In [3]:
#This notebook will explore teams by play type. Including some clustering algorthim work

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
df = pd.read_csv('../data/interim/nfl_pbp_next_scores.csv', low_memory=False)

In [4]:
#Trim to just 2019 Data for now

In [5]:
df_2019 = df.loc[df['season']==2019]

In [8]:
df_2019['play_type'].unique()

array(['run', 'pass', 'punt', 'no_play', 'field_goal', 'qb_spike'],
      dtype=object)

In [17]:
df_2019_ptypes = df_2019[['posteam','play_type','play_id']]

In [19]:
print(df_2019_ptypes.groupby(['posteam','play_type']).count())

                    play_id
posteam play_type          
ARI     field_goal       35
        no_play          77
        pass            597
        punt             70
        qb_spike          4
...                     ...
WAS     no_play          75
        pass            526
        punt             79
        qb_spike          3
        run             351

[188 rows x 1 columns]


In [20]:
df_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37766 entries, 380092 to 417857
Columns: 211 entries, Unnamed: 0 to next_score_drive
dtypes: float64(88), int64(24), object(99)
memory usage: 61.1+ MB
